In [5]:
# 누적 거래량 만들기
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

data_path = 'C:/Users/choi97201/Desktop/cjw/data/minute_month'

file_list = os.listdir(data_path)

In [6]:
volume_file_list = [curr_file for curr_file in file_list if 'volume' in  curr_file and 'stackvolume' not in curr_file]

In [7]:
for curr_volume_file in volume_file_list[-1:]:
    print(curr_volume_file)
    volume_stack_file = curr_volume_file.replace('volume', 'stackvolume')
    curr_volume_df = pd.read_csv(os.path.join(data_path, curr_volume_file))
    curr_volume_df['day'] = [str(curr_date).split(' ')[0] for curr_date in curr_volume_df['date'].tolist()]
    curr_stackvolume_df = pd.DataFrame()
    curr_stackvolume_df['date'] = curr_volume_df['date']
    
    cols = list(curr_volume_df.columns)[1:-1]
    for curr_code in tqdm(cols):
        curr_stackvolume_df[curr_code] = curr_volume_df.groupby('day')[curr_code].cumsum()
    curr_stackvolume_df.to_csv(os.path.join(data_path, volume_stack_file), index=False)

minute_2021-04_volume.csv


100%|█████████████████████████████████████████████████████████████████████████████| 2875/2875 [00:05<00:00, 500.63it/s]


In [12]:
# 3개월 분할 분봉데이터 디비에 넣기
import pandas as pd
import numpy as np
import os
from tqdm import tqdm


file_list = os.listdir(data_path)

In [16]:
curr_file = pd.read_csv(os.path.join(data_path, file_list[0]), encoding='cp949')

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
import pandas as pd
import os
from fnguide import *
import tqdm

maria = Data()
maria.setMaria(host='localhost', user='root', password='sa1234', db='ohlcv', charset='utf8')
table_list = maria.mariaShowTables()['Tables_in_ohlcv'].tolist()

min_data_path = 'C:/Users/choi97201/Desktop/cjw/data/krx_minute_month'
min_file_list = os.listdir(min_data_path)
rename_param = {'종목코드': 'code', '시가': 'open', '고가': 'high', '저가': 'low', '종가': 'close', '거래량': 'volume'}
count = 0
for file in min_file_list:
    print('[{}][{}/{}]'.format(file, count, len(min_file_list)))
    curr_df = pd.read_csv(os.path.join(min_data_path, file), encoding='cp949')[['거래일자', '시각', '종목코드', '시가', '고가', '저가', '종가', '거래량']].dropna().reset_index(drop=True)
    curr_df = curr_df.rename(columns=rename_param)
    curr_df['거래일자'] = curr_df['거래일자'].astype('str')
    print('시각을 변환합니다...')
    curr_df['시각'] = [str(curr_time).zfill(4) for curr_time in tqdm.tqdm(curr_df['시각'].tolist())]
    curr_df['시각'] = ['{}:{}:00'.format(curr_time[:2], curr_time[2:]) for curr_time in tqdm.tqdm(curr_df['시각'].tolist())]
    # print('date 형식으로 변환합니다...')
    # curr_df[['date']] = ['{}-{}-{} {}'.format(curr_df['거래일자'].iloc[i][:4], curr_df['거래일자'].iloc[i][4:6], curr_df['거래일자'].iloc[i][6:], curr_df['시각'].iloc[i]) for i in tqdm.tqdm(range(curr_df.shape[0]))]
    #     curr_df = pd.merge(curr_df, std_codes[['krx_code', 'code']], on=['krx_code'])[['code', 'date', 'open', 'high', 'low', 'close', 'volume']].dropna().reset_index(drop=True)
    curr_code_list = list(set(curr_df['code'].tolist()))
    for code in tqdm.tqdm(curr_code_list):
        curr_code_df = curr_df[curr_df['code'] == code]
        curr_code_df[['date']] = ['{}-{}-{} {}'.format(curr_code_df['거래일자'].iloc[i][:4], curr_code_df ['거래일자'].iloc[i][4:6], curr_code_df ['거래일자'].iloc[i][6:], curr_code_df ['시각'].iloc[i]) for i in range(curr_code_df .shape[0])]
        curr_code_df = curr_code_df [['date', 'open', 'high', 'low', 'close', 'volume']]
        tablename = 'a' + str(code).zfill(6) + '_min'
        if not tablename.lower() in table_list:
            cols = ['date', 'open', 'high', 'low', 'close', 'volume']
            dtypes = ['datetime', 'int', 'int', 'int', 'int', 'int']
            maria.mariaCreateTable(tablename, cols, dtypes)
            table_list.append(tablename)
        else:
            orgin_date_list = maria.mariaShowData(tablename)['date'].astype('str').tolist()
        for idx in range(curr_code_df.shape[0]):
            if curr_code_df['date'].iloc[idx] in orgin_date_list:
                continue
            maria.mariaInsertData(tablename, tuple(curr_code_df.iloc[idx]))
        maria.mariaCommitDB()
    count += 1

,거래일자,시각,시장ID,종목코드,종목명,시가(수정),고가(수정),저가(수정),종가(수정),거래량
0,20160302,900,STK,900050,중국원양자원,1574.0,1576.0,1574.0,1576.0,5696
1,20160302,901,STK,900050,중국원양자원,1576.0,1576.0,1576.0,1576.0,2437
2,20160302,902,STK,900050,중국원양자원,1576.0,1579.0,1576.0,1579.0,1552
3,20160302,903,STK,900050,중국원양자원,1576.0,1579.0,1576.0,1579.0,7541
4,20160302,904,STK,900050,중국원양자원,1579.0,1581.0,1579.0,1579.0,504
...,...,...,...,...,...,...,...,...,...,...
16923625,20160630,1650,STK,900140,코라오홀딩스,NaN,NaN,NaN,NaN,924
16923626,20160630,1710,STK,900140,코라오홀딩스,NaN,NaN,NaN,NaN,189
16923627,20160630,1740,STK,900140,코라오홀딩스,NaN,NaN,NaN,NaN,120
16923628,20160630,1750,STK,900140,코라오홀딩스,NaN,NaN,NaN,NaN,251
